[사용자 정의 모델 클래스]
- 부모클래스: nn.Module
- 필수 오버라이딩:
    - _ _init_ _(): 모델 층 구성(설계)
    - forward(): 순방향 학습 진행 코드 구현
- 동적 모델
    - container 모듈 중 nn.ModuleList() 사용해서 동적으로 layer 추가
        - forward 기능 제공 X
        - layer 인스턴스 요소 사이에 연관성 X, 인덱싱으로 접근

In [5]:
#모듈 로딩
import torch                                #텐서 관련 모듈
import torch.nn as nn                       #인공신경망 관련 모듈
import torch.nn.functional as F             #인공신경망 관렴 함수 모듈(손실함수, 활성화함수 등)
import torch.optim as optim                 #최적화 관련 모듈(가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary               #모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *       #회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   #분류 성능 지표 관련 모듈

In [6]:
#랜덤 고정
torch.manual_seed(1)

#저장 및 실행 위치 설정
device='cuda' if torch.cuda.is_available() else 'cpu'

[모델 설계-동적 모델]
- 목표: 은닉층의 개수가 동적인 모델
- 조건
    - 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값 필요
    - 은닉층의 개수 동적 + 퍼셉트론의 개수 고정 => 은닉층의 개수, 퍼셉트론 수 필요 
- - -
- 모델 이름: dynamicModel
- 부모 클래스: nn.Module
- 매개 변수: in_in, out_out, h_ins=[], h_outs=[]

In [16]:
class dynamicModel(nn.Module):
    
    #모델 구조 설계 함수(생성자 메서드)
    def __init__(self,in_in,in_out,out_in,out_out,h_ins=[],h_outs=[]):

        #부모 클래스 생성
        super().__init__()

        #설계
        self.in_layer=nn.Linear(in_in,h_ins[0] if len(h_ins) else in_out)

        self.h_layers=nn.ModuleList()
        for idx in range(len(h_ins)):
            self.h_layers.append(nn.Linear(h_ins[idx],h_outs[idx]))
        
        self.out_layer=nn.Linear(h_outs[-1] if len(h_outs) else out_in,out_out)      

    #학습 진행 콜백 메서드
    def forward(self,x):

        #입력층
        y=self.in_layer(x)          # y = x1·w1 + x2·w2 + x3·w3 + b
        y=F.relu(y)                 # 0 <= y
        # y=F.relu(self.in_layer(x))

        #은닉층
        for linear in self.h_layers:
            y=linear(y)
            y=F.relu(y)

        #출력층
        return self.out_layer(y)

In [17]:
#모델 구조 확인
h_ins_=[30,50,70]
h_outs_=[50,70,30]

# h_ins=range(10,101,20)
# h_outs=range(30,111,20)

# h_inout=[10,30,50,70,90,110]
# for idx in range(h_inout)-1:
#     nn.Linear(h_inout[idx],h_inout[idx+1])

m1=dynamicModel(3,5,5,2,h_ins=h_ins_,h_outs=h_outs_)
m1

dynamicModel(
  (in_layer): Linear(in_features=3, out_features=30, bias=True)
  (h_layers): ModuleList(
    (0): Linear(in_features=30, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=70, bias=True)
    (2): Linear(in_features=70, out_features=30, bias=True)
  )
  (out_layer): Linear(in_features=30, out_features=2, bias=True)
)

In [12]:
#모델 파라미터 확인
for name,param in m1.named_parameters():
    print(name,param.shape)

in_layer.weight torch.Size([5, 3])
in_layer.bias torch.Size([5])
out_layer.weight torch.Size([2, 5])
out_layer.bias torch.Size([2])


<!-- [학습 진행] -->

In [16]:
#임의의 데이터 생성
dataTS=torch.FloatTensor([[1,3,5],[2,4,6],[3,5,7],[4,6,8]])     #2D (4,3)
targetTS=torch.FloatTensor([[7,9],[8,10],[9,11],[10,12]])       #2D (4,2)

In [18]:
#모델 학습
pre_y=m1(dataTS)
print(pre_y,pre_y.shape,sep='\n')

tensor([[0.1237, 0.0215],
        [0.1237, 0.0215],
        [0.1237, 0.0215],
        [0.1237, 0.0215]], grad_fn=<AddmmBackward0>)
torch.Size([4, 2])
